In [2]:
from darts.datasets import AirPassengersDataset, AusBeerDataset, ETTh1Dataset, ETTh2Dataset, ETTm1Dataset, ETTm2Dataset, ExchangeRateDataset, TrafficDataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

np.random.seed(123)



In [9]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset

torch.set_default_dtype(torch.float64)

In [53]:


class ESN(nn.Module):

    def __init__(self, reservoir_size=100, input_size=1, output_size=1,  spectral_radius=1.0, connectivity_rate=1.0, learning_rate = 0.1, epochs=1, washout=1, activation="tanh"):
        super(ESN, self).__init__()
        self.reservoir_size = reservoir_size
        self.input_size = input_size
        self.epochs = epochs
        self.connectivity_rate = connectivity_rate
        self.spectral_radius = spectral_radius
        self.washout = washout
        self.output_size = output_size
        self.lr = learning_rate
        self.activation = self.activation_fn(activation)
        

        self.state = torch.zeros(self.reservoir_size, 1)
        self.W_in = torch.rand((reservoir_size, input_size)) * 2 - 1
        
        
        self.W_out = None

        ## Initializing Reservoir Weights according to "Re-visiting the echo state property"(2012)
        ##
        ## Initialize a random matrix and induce sparsity.
        self.W_res = torch.rand((reservoir_size, reservoir_size))
        self.W_res.data[torch.rand(*self.W_res.shape) > self.connectivity_rate] = 0

        ## Scale the matrix based on user defined spectral radius.
        current_spectral_radius = torch.max(torch.abs(torch.linalg.eigvals(self.W_res.data)))
        self.W_res.data = self.W_res * (self.spectral_radius / current_spectral_radius)

        ## Induce half of the weights as negative weights.
        total_entries = self.reservoir_size * self.reservoir_size
        num_negative_entries = total_entries//2
        negative_indices = np.random.choice(total_entries, num_negative_entries, replace=False)
        W_flat = self.W_res.data.flatten()
        W_flat[negative_indices] *= -1
        self.W_res = W_flat.reshape(*self.W_res.shape)


        self.all_states = [self.state]

    @staticmethod
    def activation_fn(x):
         
        activation_keys = ["sigmoid", "relu", "tanh"]

        if x in activation_keys:
              if x == "tanh":
                   return nn.Tanh()
              elif x == "relu":
                   return nn.ReLU()
              elif x == "sigmoid":
                   return nn.Sigmoid()
            
        else:
            raise ValueError(f"Activation {x} does not exists")
    



    def fit(self, X_train, y_train):
        

        state_collection_matrix = torch.zeros(self.input_size + self.reservoir_size, 1)
        # self.state = np.zeros((self.reservoir_size, 1))

        ## Calculate state of reservoirs per time step
        for i in range(X_train.shape[0]-1):
            input = X_train[i].reshape(-1,1)
            input_product = self.W_in@input
            state_product = self.W_res@self.state
            self.state = self.activation(input_product + state_product)
            state_collection_matrix= torch.hstack((state_collection_matrix, torch.concatenate((self.state, input))))

            self.all_states.append(self.state)

        ## Update W_out
        mat1 = state_collection_matrix.T[self.washout:,:]
        X = torch.matmul(mat1.T, mat1)

        Y =torch.matmul(mat1.T, y_train[self.washout:,:])

        eyes = torch.eye(self.reservoir_size)
        print(eyes.shape)

        print(X.shape, Y.shape)
        self.W_out = torch.matmul(torch.linalg.inv(X + self.lr * eyes, Y))

    
    def predict(self, X_test):
            prediction = np.zeros((self.output_size,1))
            for i in range(X_test.shape[0]- 1):
                input = X_test[i].reshape(-1,1)
                input_product = self.W_in@input
                state_product = self.W_res@self.state
                self.state = self.activation(input_product + state_product)
                concat_matrix= torch.concatenate((self.state, input))
                pred =  self.W_out@concat_matrix
                prediction = torch.hstack([prediction, pred])

                self.all_states.append(self.state)
            
            prediction = rearrange(prediction, 'c r -> r c')
            if self.output_size == self.input_size:
                return prediction[1:,:]
            else:
                return prediction
            

In [54]:
uni_to_uni_datasets = [{"name": "AirPassengers",
             "dataset": AirPassengersDataset(),
             "input": 1,
             "output": 1},
            {"name":"AusBeer",
             "dataset": AusBeerDataset(),
             "input": 1,
             "output": 1}
            ]


multi_to_uni_datasets = [{"name": "ETTh1",
                   "dataset": ETTh1Dataset(),
                   "input": 6,
                   "output": 1},
                   {"name": "ETTh2",
                   "dataset": ETTh2Dataset(),
                   "input": 6,
                   "output": 1},
                   {"name": "ETTm1",
                   "dataset": ETTm1Dataset(),
                   "input": 6,
                   "output": 1},
                   {"name": "ETTm2",
                   "dataset": ETTm2Dataset(),
                   "input": 6,
                   "output": 1}]

multi_to_multi_datasets = [{"name": "ETTh1",
                   "dataset": ExchangeRateDataset(),
                   "input": 8,
                   "output": 8}]




metrics = []

In [55]:
test_size = 0.2
reservoir_size = 20
spectral_radius = 0.7
connectivity_rate = 0.8
washout = 1

In [56]:
data = uni_to_uni_datasets[0]["dataset"]
time_series = data.load()
X = time_series.values()
X_train, X_test = train_test_split(X, test_size=test_size, shuffle=False)

sc = MinMaxScaler()
X_train_std = torch.tensor(sc.fit_transform(X_train))
X_test_std = torch.tensor(sc.transform(X_test))

esn = ESN(reservoir_size=reservoir_size, input_size=1, output_size=1, spectral_radius=spectral_radius, connectivity_rate=connectivity_rate, washout=washout)
esn.fit(X_train_std, X_train_std)



torch.Size([20, 20])
torch.Size([21, 21]) torch.Size([21, 1])


RuntimeError: The size of tensor a (21) must match the size of tensor b (20) at non-singleton dimension 1

In [45]:


for i in uni_to_uni_datasets:
    data = i["dataset"]
    input_size = i["input"]
    output_size = i["output"]
    name = i["name"]

    time_series = data.load()
    X = time_series.values()
    X_train, X_test = train_test_split(X, test_size=test_size, shuffle=False)

    sc = MinMaxScaler()
    X_train_std = sc.fit_transform(X_train)
    X_test_std = sc.transform(X_test)


    esn = ESN(reservoir_size=reservoir_size, input_size=input_size, output_size=output_size, spectral_radius=spectral_radius, connectivity_rate=connectivity_rate, washout=washout)
    esn.fit(X_train_std, X_train_std)

    test_values= X_test_std[1:]

    predictions = esn.predict(X_test_std)

    metrics.append({"Data": name,
                    "RMSE": np.sqrt(mean_squared_error(test_values, predictions)),
                    "MAE" : mean_absolute_error(test_values, predictions),
                    "MAPE": mean_absolute_percentage_error(test_values, predictions)       
                    })

In [46]:

for i in multi_to_uni_datasets:
    data = i["dataset"]
    input_size = i["input"]
    output_size = i["output"]
    name = i["name"]

    time_series = data.load().values()
    X = time_series[:,:-1]
    y = time_series[:,-1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)


    y_train = rearrange(y_train, 'r -> r 1')
    y_test = rearrange(y_test, 'r -> r 1')


    sc1 = MinMaxScaler()
    X_train_std = sc1.fit_transform(X_train)
    X_test_std = sc1.transform(X_test)

    sc2 = MinMaxScaler()
    y_train_std = sc2.fit_transform(y_train)
    y_test_std = sc2.transform(y_test)
    

    esn = ESN(reservoir_size=reservoir_size, input_size=input_size, output_size=output_size, spectral_radius=spectral_radius, connectivity_rate=connectivity_rate, washout=washout)
    esn.fit(X_train_std, y_train_std)

    predictions = esn.predict2(X_test_std)

    metrics.append({"Data": name,
                    "RMSE": np.sqrt(mean_squared_error(y_test_std, predictions)),
                    "MAE" : mean_absolute_error(y_test_std, predictions),
                    "MAPE": mean_absolute_percentage_error(y_test_std, predictions)       
                    })



In [47]:
for i in multi_to_multi_datasets:
    data = i["dataset"]
    input_size = i["input"]
    output_size = i["output"]
    name = i["name"]

    time_series = data.load()
    X = time_series.values()
    X_train, X_test = train_test_split(X, test_size=test_size, shuffle=False)

    sc = MinMaxScaler()
    X_train_std = sc.fit_transform(X_train)
    X_test_std = sc.transform(X_test)


    esn = ESN(reservoir_size=reservoir_size, input_size=input_size, output_size=output_size, spectral_radius=spectral_radius, connectivity_rate=connectivity_rate, washout=washout)
    esn.fit(X_train_std, X_train_std)

    test_values= X_test_std[1:]

    predictions = esn.predict(X_test_std)

    metrics.append({"Data": name,
                    "RMSE": np.sqrt(mean_squared_error(y_test_std, predictions)),
                    "MAE" : mean_absolute_error(y_test_std, predictions),
                    "MAPE": mean_absolute_percentage_error(y_test_std, predictions)       
                    })

ValueError: Found input variables with inconsistent numbers of samples: [13936, 1517]

In [ ]:
df = pd.DataFrame(metrics)
df

In [ ]:
print(df)